<a href="https://colab.research.google.com/github/m-newhauser/rep-or-dem-tweets/blob/main/finetune_full_architecture_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resources

* [Fine-tuning DistilBERT with TF (freezing last hidden layer from DistilBERT models)](https://towardsdatascience.com/hugging-face-transformers-fine-tuning-distilbert-for-binary-classification-tasks-490f1d192379)
* [Fine-tuning DistilBERT with only TF](https://medium.com/geekculture/hugging-face-distilbert-tensorflow-for-custom-text-classification-1ad4a49e26a7)

In [23]:
!pip install transformers==4.6.0
!pip install tweet-preprocessor

In [24]:
import random
import pandas as pd
import numpy as np
import csv
import tensorflow as tf
import preprocessor as p

from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import (
    TFDistilBertForSequenceClassification,
    TFTrainer,
    TFTrainingArguments,
)

from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

random.seed(123)

## Pre-process data

In [25]:
# Load data from fivethirtyeight
tweets = pd.read_csv("senators_training.csv")
tweets_validation = pd.read_csv("senators_validation.csv")

# Remove numbers, emojis and &'s
p.set_options(p.OPT.NUMBER, p.OPT.EMOJI)
tweets["text_clean"] = tweets["text"].apply(p.clean)
tweets["text_clean"] = tweets["text_clean"].str.replace("&amp;", "and ")
tweets_validation["text_clean"] = tweets_validation["text"].apply(p.clean)
tweets_validation["text_clean"] = tweets_validation["text_clean"].str.replace("&amp;", "and ")

# Truncate tweets to max 512
tweets["text_clean"] = tweets["text_clean"].str[:512]
tweets_validation["text_clean"] = tweets_validation["text_clean"].str[:512]


In [26]:
# Create a column with numeric labels
label_mapping = {
    "D": 0,
    "R": 1
}

tweets['label'] = np.where(tweets['party'] == "D", 0, 1)
tweets_validation['label'] = np.where(tweets_validation['party'] == "D", 0, 1)

In [27]:
# Convert to list
texts = list(tweets.text)
labels = list(tweets.label)
val_texts = list(tweets_validation.text)
val_labels = list(tweets_validation.label)

# Split training dataset into test and train
(train_texts, test_texts, train_labels, test_labels) = train_test_split(
    texts, labels, test_size=0.3
)

### Tokenize data for DistilBERT

In [28]:
# Load DistilBERT tokenizer and tokenize (encode) the texts
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)


### Create encodings

In [29]:
# Wrap encodings in a Tensor Flow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

## Fine-tune entire DistilBERT architecture (layers)

## Add a TF classifier head

In [30]:
# TF parameters
EPOCHS = 6
BATCH_SIZE = 64
# LAYER_DROPOUT = 0.2

num_train_steps = len(train_dataset) * EPOCHS

# Define TF classifier model components and compile
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)
optimizer = Adam(learning_rate=lr_scheduler)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

# Train the model 
model.fit(train_dataset.shuffle(len(train_dataset)).batch(BATCH_SIZE),
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          )

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_projector', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_59']
You should probably TRAIN this model on a down-stream task to be able to use i

Epoch 1/6
8/8 [==============================] - 169s 19s/step - loss: 0.6955 - accuracy: 0.5184
Epoch 2/6
8/8 [==============================] - 155s 19s/step - loss: 0.6902 - accuracy: 0.5265
Epoch 3/6
8/8 [==============================] - 156s 19s/step - loss: 0.6619 - accuracy: 0.6469
Epoch 4/6
8/8 [==============================] - 157s 20s/step - loss: 0.5525 - accuracy: 0.7673
Epoch 5/6
8/8 [==============================] - 156s 19s/step - loss: 0.3406 - accuracy: 0.8735
Epoch 6/6
8/8 [==============================] - 157s 20s/step - loss: 0.1709 - accuracy: 0.9490


In [31]:
# Prepare the validation dataset for predictions
val_dataset_finetune = val_dataset.shuffle(len(test_dataset)).batch(32).repeat(2)

# Make predictions and evaluate
model.evaluate(val_dataset_finetune)

20/20 [==============================] - 74s 4s/step - loss: 1.0640 - accuracy: 0.6200


[1.0640172958374023, 0.6200000047683716]